# Sequence to Sequence model to extract summary,objective from resume (input: resume line by line)


## 207 resumes


In [87]:
from __future__ import print_function
import csv,os
import collections
from keras.models import load_model,Sequential
from keras.layers import Input,LSTM,Dense,Activation
from keras.callbacks import ModelCheckpoint
import numpy as np

batch_size=128
epochs=20
latent_dim=128
data_path="/home/santhosh/resumes_folder/custom_annotator/annotator-server/static/files/Data_Tracter_Resumes_in_TXT/csv"

## vectorizing the data

In [88]:
input_resumes=[]
output_summary=[]
input_tokens=collections.Counter()
output_tokens=set()
files=os.listdir(data_path)
count=0
for file in files[:50]:
    with open(data_path+'/'+file,'r') as csv_file:
        reader=csv.reader(csv_file)
        count=0
        for Input_text,output_text in reader:
            Input_text=Input_text.strip().lower()
            Input_text=Input_text
            
            input_resumes.append(Input_text)
            output_summary.append(output_text)
            
            for word in Input_text.split():
                if word not in input_tokens:
                    input_tokens[word]+=1
            for word in output_text.split():
                if word not in output_tokens:
                    output_tokens.add(word)
            if count==50:
                break
            count+=1


In [89]:
print(len(input_tokens))
num_encoder_tokens=min(len(input_tokens),4000)
input_tokens=[word for word,count in input_tokens.most_common(num_encoder_tokens-1)]
input_tokens=sorted(list(input_tokens))
output_tokens=sorted(list(output_tokens))
num_decoder_tokens=len(output_tokens)
max_encoder_seq_len=max([len(text.split()) for text in input_resumes])
max_decoder_seq_len=max([len(text.split()) for text in output_summary])

print('Number of samples:',len(input_resumes))
print('number of unique input token:',num_encoder_tokens)
print('number of unique output token:',num_decoder_tokens)
print('Max Sequence length for inputs:',max_encoder_seq_len)
print('Max Sequence length for outputs:',max_decoder_seq_len)


5179
Number of samples: 2510
number of unique input token: 4000
number of unique output token: 2
Max Sequence length for inputs: 24
Max Sequence length for outputs: 1


## defining token2index

In [90]:
input_token2index=dict([(word,i) for i,word in enumerate(input_tokens)])
output_token2index=dict([(word,i) for i,word in enumerate(output_tokens)])
output_index2token=dict([(i,word) for i,word in enumerate(output_tokens)])
input_token2index['UNK']=num_encoder_tokens-1

In [91]:
output_index2token

{0: '0', 1: '1'}

## defing encoder_input,decoder_input and decoder_output

In [92]:
encoder_input_data=np.zeros((len(input_resumes),max_encoder_seq_len,num_encoder_tokens),dtype='float32')
decoder_target_data=np.zeros((len(input_resumes),num_decoder_tokens),dtype='float32')


## creating training dataset

In [93]:
for i,(input_text,target_text) in enumerate(zip(input_resumes,output_summary)):
    for t,word in enumerate(input_text.split()[:max_encoder_seq_len]):
        if word not in input_token2index:
            word="UNK"
        encoder_input_data[i,t,input_token2index[word]]=1
    decoder_target_data[i,output_token2index[target_text]]=1
        

In [94]:
model = Sequential()
model.add(LSTM(latent_dim, return_sequences=False,
input_shape=(max_encoder_seq_len,num_encoder_tokens,)))
model.add(Dense(num_decoder_tokens))
model.add(Activation("softmax"))
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics=["accuracy"])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_9 (LSTM)                (None, 128)               2114048   
_________________________________________________________________
dense_7 (Dense)              (None, 2)                 258       
_________________________________________________________________
activation_6 (Activation)    (None, 2)                 0         
Total params: 2,114,306
Trainable params: 2,114,306
Non-trainable params: 0
_________________________________________________________________


## function to test model

## Run training

In [95]:

iteration=0
"""
# load weights
print('loading the weights')
model=load_model('resume_level.h5')

# estimate accuracy on whole dataset using loaded weights
scores = model.evaluate([encoder_input_data, decoder_input_data], decoder_target_data,verbose=0)
print("%s: %.2f%%\n\n" % (model.metrics_names[1], scores[1]*100))
print("Testing Samples\n"+"-"*50)
for i in range(1):
    index=np.random.randint(len(input_resumes))
    encoded_input_sequence=encoder_input_data[index: index + 1]
    output_sequence=decode_sequence(encoded_input_sequence)
    print("-"*50)
    print(input_resumes[index])
    print(" "*50)
    print("*"*50+"\nOUTPUT"+" "*50)
    print(output_sequence)
    print("-"*50+"\n"+" "*50)
"""
iteration_file="/home/santhosh/resumes_folder/keras/extract_summary_and_objective/iteration_resume_line_classification.txt"
try:
    file=open(iteration_file,'r')
    last_line=file.read().split('\n')[-2]
    print('file_data,',last_line)
    iteration=int(last_line.split(':')[1])
    #print(iteration)
    file.close()
    
except:
    print('no file exist')

# checkpoint
filepath="resume_line_classification_checkpoints.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=0, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

while True:
    print('Iteration:',iteration+1)
    #training
    model.fit(encoder_input_data,decoder_target_data ,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2,callbacks=callbacks_list)
    
    #prepare sample_data to test 5 samples:
    print("-"*50)
    index=int(np.random.randint(len(input_resumes)/40*0.8)*40)
    test_input=""
    test_output=""
    for i in range(50):
        
        encoded_input_sequence=encoder_input_data[index: index + 1]
        output_sequence=model.predict(encoded_input_sequence, verbose=0)[0]
        output_sequence = output_index2token[np.argmax(output_sequence)]
        test_input+=input_resumes[index]+'\n'
        if output_sequence=="1":
            output_sequence=input_resumes[index]+'\n'
        else:
            output_sequence=''
        test_output+=output_sequence
        index+=1
    print("-"*50)
    print(test_input)
    print("---OUTPUT-----")
    print(test_output)
    print(" "*50+"-"*50)
        
        
        
    # Save model
    file=open(iteration_file,'a')
    file.write('iteration:'+str(iteration+1)+'\n')
    file.close()
    iteration+=1
    model.save('resume_line_classification.h5')

file_data, iteration:3
Iteration: 4
Train on 2008 samples, validate on 502 samples
Epoch 1/20
2008/2008 [==============================] - 50s 25ms/step - loss: 0.5835 - acc: 0.7545 - val_loss: 0.4561 - val_acc: 0.8167
Epoch 2/20
2008/2008 [==============================] - 47s 23ms/step - loss: 0.4123 - acc: 0.8152 - val_loss: 0.4020 - val_acc: 0.9024
Epoch 3/20
2008/2008 [==============================] - 47s 23ms/step - loss: 0.2601 - acc: 0.8914 - val_loss: 0.3077 - val_acc: 0.8964
Epoch 4/20
2008/2008 [==============================] - 50s 25ms/step - loss: 0.2195 - acc: 0.9148 - val_loss: 0.3227 - val_acc: 0.9004
Epoch 5/20
2008/2008 [==============================] - 50s 25ms/step - loss: 0.1521 - acc: 0.9437 - val_loss: 0.3376 - val_acc: 0.8805
Epoch 6/20
2008/2008 [==============================] - 53s 27ms/step - loss: 0.1417 - acc: 0.9512 - val_loss: 0.4543 - val_acc: 0.7629
Epoch 7/20
2008/2008 [==============================] - 47s 23ms/step - loss: 0.1137 - acc: 0.9597 - 

Iteration: 6
Train on 2008 samples, validate on 502 samples
Epoch 1/20
2008/2008 [==============================] - 50s 25ms/step - loss: 0.0276 - acc: 0.9905 - val_loss: 0.7324 - val_acc: 0.8426
Epoch 2/20
2008/2008 [==============================] - 50s 25ms/step - loss: 0.0297 - acc: 0.9895 - val_loss: 1.0198 - val_acc: 0.8347
Epoch 3/20
2008/2008 [==============================] - 50s 25ms/step - loss: 0.0298 - acc: 0.9885 - val_loss: 1.2313 - val_acc: 0.8167
Epoch 4/20
2008/2008 [==============================] - 46s 23ms/step - loss: 0.0285 - acc: 0.9920 - val_loss: 0.9279 - val_acc: 0.8207
Epoch 5/20
2008/2008 [==============================] - 46s 23ms/step - loss: 0.0271 - acc: 0.9910 - val_loss: 0.9621 - val_acc: 0.7869
Epoch 6/20
2008/2008 [==============================] - 46s 23ms/step - loss: 0.0295 - acc: 0.9925 - val_loss: 0.6974 - val_acc: 0.8048
Epoch 7/20
2008/2008 [==============================] - 46s 23ms/step - loss: 0.0277 - acc: 0.9890 - val_loss: 0.6671 - val_

Epoch 16/20
2008/2008 [==============================] - 47s 23ms/step - loss: 0.0261 - acc: 0.9910 - val_loss: 1.1058 - val_acc: 0.8207
Epoch 17/20
2008/2008 [==============================] - 48s 24ms/step - loss: 0.0207 - acc: 0.9920 - val_loss: 1.2589 - val_acc: 0.8227
Epoch 18/20
2008/2008 [==============================] - 47s 23ms/step - loss: 0.0225 - acc: 0.9920 - val_loss: 1.1124 - val_acc: 0.8307
Epoch 19/20
2008/2008 [==============================] - 46s 23ms/step - loss: 0.0221 - acc: 0.9925 - val_loss: 1.2314 - val_acc: 0.8267
Epoch 20/20
2008/2008 [==============================] - 48s 24ms/step - loss: 0.0468 - acc: 0.9900 - val_loss: 1.1764 - val_acc: 0.8147
--------------------------------------------------
--------------------------------------------------
 carrying 3.10 years of qualitative experience in software testing with key strengths in web/mobile
application testing (android, ios, blackberry and windows).
 presently associated with tata consultancy service

--------------------------------------------------
--------------------------------------------------
publication:
book name
e-governance in india - priorities, challenges & benefits” published by lambert academic
publishing gmbh & co. in october 12, 2011
summary
my publication is review of current trends and strategies in the area of e-governance. in the current
democracies, there is an increasing demand to bring citizens closer to the government and to improve
delivery of government services.
professional awards:
star awards
 recognized as an individual contributor for outstanding quarterly technical excellence contributing to
a research & innovation of new product, cross-group technology transfer, mentorship “above and
b
b.aravindhan
email: b.aravind87@gmail.com
mobile: +91 7406006123
career objective:
seeking a excellent and challenging career in an information and communication technology organization
where i can show my leadership and teamwork skills that would mold me into a dy

2008/2008 [==============================] - 35s 17ms/step - loss: 0.0199 - acc: 0.9925 - val_loss: 1.4571 - val_acc: 0.8247
Epoch 2/20


KeyboardInterrupt: 